In [14]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount("/content/gdrive",force_remount=True)

Mounted at /content/gdrive


In [15]:
import os
os.chdir("/content/gdrive/MyDrive/CS444/assignment3_part2/")

In [16]:
import os
import random

import cv2
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import models

from src.resnet_yolo import resnet50
from yolo_loss import YoloLoss
from src.dataset import VocDetectorDataset
from src.eval_voc import evaluate
from src.predict import predict_image
from src.config import VOC_CLASSES, COLORS
from kaggle_submission import output_submission_csv

import matplotlib.pyplot as plt
import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialization

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
load_network_path = 'checkpoints/best_detector.pth' #'checkpoints/best_detector.pth' 
pretrained = True

# use to load a previously trained network
if load_network_path is not None:
    print('Loading saved network from {}'.format(load_network_path))
    net = resnet50().to(device)
    net.load_state_dict(torch.load(load_network_path))
else:
    print('Load pre-trained model')
    net = resnet50(pretrained=pretrained).to(device)

Loading saved network from checkpoints/best_detector.pth


# Video predictions

In [17]:
# Open video file using OpenCV
video_path = 'SNL Digital Short- YOLO - SNL.mp4'
cap = cv2.VideoCapture(video_path)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
middle_frame = num_frames // 1.5
cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)

True

In [18]:
# Define video writer object
output_path = 'output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [19]:
# Loop through video frames
net.eval()

# Process each frame in the video
for i in range(1,500):
    # Read the frame from the video
    ret, frame = cap.read()

    if not ret:
        break

    # Save the input frame to disk
    input_path = os.path.join('input_frames', f'frame_{i}.jpg')
    cv2.imwrite(input_path, frame)


    image = cv2.imread(input_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # print('predicting...')
    print(f'frame_{i}.jpg')
    result = predict_image(net,f'frame_{i}.jpg',root_img_directory ='input_frames/')
    for left_up, right_bottom, class_name, _, prob in result:
      color = COLORS[VOC_CLASSES.index(class_name)]
      cv2.rectangle(image, left_up, right_bottom, color, 2)
      label = class_name + str(round(prob, 2))
      text_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
      p1 = (left_up[0], left_up[1] - text_size[1])
      cv2.rectangle(image, (p1[0] - 2 // 2, p1[1] - 2 - baseline), (p1[0] + text_size[0], p1[1] + text_size[1]),
                    color, -1)
      cv2.putText(image, label, (p1[0], p1[1] + baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, 8)

    # Save the output frame to disk
    output_path = os.path.join('output_frames', f'frame_{i}.jpg')
    cv2.imwrite(output_path, image)

    out.write(image)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

frame_1.jpg
frame_2.jpg
frame_3.jpg
frame_4.jpg
frame_5.jpg
frame_6.jpg
frame_7.jpg
frame_8.jpg
frame_9.jpg
frame_10.jpg
frame_11.jpg
frame_12.jpg
frame_13.jpg
frame_14.jpg
frame_15.jpg
frame_16.jpg
frame_17.jpg
frame_18.jpg
frame_19.jpg
frame_20.jpg
frame_21.jpg
frame_22.jpg
frame_23.jpg
frame_24.jpg
frame_25.jpg
frame_26.jpg
frame_27.jpg
frame_28.jpg
frame_29.jpg
frame_30.jpg
frame_31.jpg
frame_32.jpg
frame_33.jpg
frame_34.jpg
frame_35.jpg
frame_36.jpg
frame_37.jpg
frame_38.jpg
frame_39.jpg
frame_40.jpg
frame_41.jpg
frame_42.jpg
frame_43.jpg
frame_44.jpg
frame_45.jpg
frame_46.jpg
frame_47.jpg
frame_48.jpg
frame_49.jpg
frame_50.jpg
frame_51.jpg
frame_52.jpg
frame_53.jpg
frame_54.jpg
frame_55.jpg
frame_56.jpg
frame_57.jpg
frame_58.jpg
frame_59.jpg
frame_60.jpg
frame_61.jpg
frame_62.jpg
frame_63.jpg
frame_64.jpg
frame_65.jpg
frame_66.jpg
frame_67.jpg
frame_68.jpg
frame_69.jpg
frame_70.jpg
frame_71.jpg
frame_72.jpg
frame_73.jpg
frame_74.jpg
frame_75.jpg
frame_76.jpg
frame_77.jpg
frame_78